### Work in progress - samla in Alvins yrken

In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2021-08-15 08:56:35.872580


In [2]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys,json
import pandas as pd 

from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

queryAlvin = """SELECT ?item ?itemLabel ?alvinid WHERE {
  ?item wdt:P6821 ?alvinid.
  ?item wdt:P31 wd:Q5.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv". }
} limit 10000"""

def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDAlvintot = get_sparql_dataframe(endpoint_url, queryAlvin)
WDAlvintot["Source"] = "Alvin"

In [3]:
WDAlvintot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   item       100 non-null    object
 1   itemLabel  100 non-null    object
 2   alvinid    100 non-null    object
 3   Source     100 non-null    object
dtypes: object(4)
memory usage: 3.2+ KB


In [4]:
WDAlvintot.head(5)

,item,itemLabel,alvinid,Source
0,http://www.wikidata.org/entity/Q297,Diego Velázquez,alvin-person:61604,Alvin
1,http://www.wikidata.org/entity/Q619,Nicolaus Copernicus,alvin-person:12091,Alvin
2,http://www.wikidata.org/entity/Q747,Pierre Corneille,alvin-person:67256,Alvin
3,http://www.wikidata.org/entity/Q859,Platon,alvin-person:29707,Alvin
4,http://www.wikidata.org/entity/Q859,Platon,alvin-person:69659,Alvin


In [14]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup, NavigableString, Tag
def cleanupstring(s):
    out_s = s
    while '  ' in out_s:
        out_s = out_s.strip().replace('  ', ' ')
    return out_s, len(s)-len(out_s)


newList = []
urlBase = "https://www.alvin-portal.org/alvin/view.jsf?pid="
for index, row in WDAlvintot.iterrows():
    alvinid = row['alvinid']
    person = row['itemLabel']
    url = urlBase + alvinid
    #print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    for header in soup.find_all('h2'): 
        nextNode = header
        if "Occupation" in header:
            while True:
            
                nextNode = nextNode.nextSibling
                if nextNode is None:
                    break
                if isinstance(nextNode, NavigableString):
                    print (alvinid,"-",person,"\tyrke: ",nextNode.strip())
                    newList.append(nextNode.strip())
                    break
#                if isinstance(nextNode, Tag):
#                    if nextNode.name == "h2":
#                        break
                #print (nextNode.get_text(strip=True).strip())

print("Lines found:\t", len(newList))    
# TBD
    

alvin-person:61604 - Diego Velázquez 	yrke:  Konstnär
alvin-person:12091 - Nicolaus Copernicus 	yrke:  Astronomer
alvin-person:67256 - Pierre Corneille 	yrke:  Advokat, Kompositör, Författare, Tonsättare
alvin-person:29707 - Platon 	yrke:  filosof
alvin-person:34553 - Ban Ki-moon 	yrke:  Förenta nationernas generalsekreterare
alvin-person:15458 - Richard Wagner 	yrke:  Composer, kompositör
alvin-person:10891 - Albrecht Dürer 	yrke:  Engraver, Painter
alvin-person:56168 - Quintus Curtius Rufus 	yrke:  historiker
alvin-person:1063 - Marie Curie 	yrke:  Fysiker, Radiolog, Kemist, Chemist, Physicist, kemist, fysiker
alvin-person:48342 - Simone de Beauvoir 	yrke:  författare, filosof, feminist
alvin-person:27012 - Ingmar Bergman 	yrke:  Regissör, Författare, regissör
alvin-person:64330 - Grazia Deledda 	yrke:  Författarinna
alvin-person:25934 - Arkimedes 	yrke:  Matematiker
alvin-person:10603 - Augustin Louis Cauchy 	yrke:  Mathematician, Engineer
alvin-person:3534 - Martin Luther 	yrke:  T

In [51]:
import pandas as pd
newListdf = pd.DataFrame(newList,columns=list(["Yrke"]))

In [54]:
newListdf

,Yrke
0,Konstnär
1,Astronomer
2,"Advokat, Kompositör, Författare, Tonsättare"
3,filosof
4,Förenta nationernas generalsekreterare
...,...
64,författare
65,Kurfurste
66,"Apothecary, Chemist"
67,"Internist, Physician"


In [55]:
yrkendf = pd.concat([pd.Series(row['Yrke'].split(','))              
                    for _, row in newListdf.iterrows()]).reset_index() 
yrkendf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   139 non-null    int64 
 1   0       139 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.3+ KB


In [57]:
mapping = {yrkendf.columns[1]:'yrke'}
yrkendf = yrkendf.rename(columns=mapping)
yrkendf["yrke"] = yrkendf['yrke'].str.strip()
yrkenAlvin = yrkendf["yrke"].value_counts() 

In [ ]:
yrkenAlvin.